In [14]:
!pip install webdriver-manager


# InstaBot
Instagram bot and scrapper with the finality of winning more followers

In [1]:

import random
import sys
import time
import os
import csv

import pandas as pd


import Credential

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

from tqdm import tqdm



In [2]:
#
#pip install webdriver-manager
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240


[WDM] - Driver [/home/aingeru/.wdm/drivers/chromedriver/linux64/86.0.4240.22/chromedriver] found in cache


# Login 

In [3]:
def login(driver, username, password):
    driver.get("https://www.instagram.com/")
    time.sleep(2)
    # login_button = driver.find_element_by_xpath("//a[@href='/accounts/login/?source=auth_switcher']")
    # login_button.click()
    # time.sleep(2)
    user_name_elem = driver.find_element_by_xpath("//input[@name='username']")
    user_name_elem.clear()
    user_name_elem.send_keys(username)
    password_elem = driver.find_element_by_xpath("//input[@name='password']")
    password_elem.clear()
    password_elem.send_keys(password)
    password_elem.send_keys(Keys.RETURN)
    time.sleep(2)
    


In [4]:
# driver = webdriver.Chrome('./chromedriver')
username = Credential.username
password = Credential.password
login(driver, username, password)

# Like comments
Like all comments in a post

In [92]:
'''
Action blocked html 
<div class="pbNvD  fPMEg   " role="dialog">
    <div class="piCib">
        <div class="_08v79">
            <h3 class="_7UhW9     LjQVu     qyrsm KV-D4        uL8Hv         ">
                Action Blocked
            </h3>
            <div class="_7UhW9   xLCgt      MMzan   _0PwGv       uL8Hv         ">
                This action was blocked. Please try again later. We restrict certain content and actions to protect our community. Tell us if you think we made a mistake.</div></div><div class="mt3GC"><button class="aOOlW  bIiDR  " tabindex="0">Report a Problem</button><button class="aOOlW   HoLwm " tabindex="0">OK</button></div></div></div>
'''
def deal_blocked_action():
    div = driver.find_element_by_class_name('mt3GC')
    btn = div.find_elements_by_tag_name('button')
    button_ok = btn[1].click()

In [97]:
def __load_comments(n, delay=2):
    '''Click the watch more comments in the post n times'''
    print(f'There are {n} comments, loading them...')
    n = n // 12  # 12 comments updated at time
    for _ in tqdm(range(n), total=n, position=0, leave=True):  # force the tqdm bar to stay in the same line with position=0 and leave=True
        try:
            btn = driver.find_element_by_xpath("//button[@class='dCJp8 afkep']")
        except NoSuchElementException as e:
            break  # we have all the comments loaded
        btn.click()
        time.sleep(delay)

def __love_comment(comment):
    svgs = comment.find_elements_by_tag_name('svg')
    for svg in svgs:
        attr = svg.get_attribute('aria-label')
        if attr == 'Unlike':  # we've loved the button
            return True
        elif attr == 'Like':
            try:
                svg.click()
            except Exception as e:
                print("[!] IG got angry man, stop")  # love click limit reached
                return False
            return True
        
def __get_nick_from_comment(ul_comment):
    h3 = ul_comment.find_element_by_tag_name('h3')
    a = h3.find_element_by_tag_name('a')
    return a.text

def __load_previous_nicknames_loved():
    if os.path.exists('loved.txt'):  # we have loved previous comments
        fname = 'loved.txt'
        with open(fname, 'r') as fp:
            loved = set(x for x in fp.read().split('\n'))
        loved.remove('') 
        print(f'{len(loved)} already loved')
        return loved
    return None
    
def like_comments(url, comments_count, loaded=False):
    driver.get(url)
    time.sleep(2)
    if not loaded:
        __load_comments(comments_count)
    print("[OK] All comments in the screen")
    loved_hist = __load_previous_nicknames_loved()
    comments = driver.find_elements_by_class_name('Mr508')
    ul_comments = filter(lambda c: c.tag_name == 'ul', comments)
    with open('loved.txt', 'a+') as fp:
        for comment in tqdm(ul_comments, total=comments_count, leave=True, position=0 ):
            nick = __get_nick_from_comment(comment)
            if loved_hist and nick in loved_hist:
                continue
            if not __love_comment(comment):
                return False  # IG is upset and thinks I'm a bot haha
            fp.write(nick + '\n')
            time.sleep(random.randint(1, 8))  # delay randomly to avoid anti-bot algorithm
    return True


In [98]:
# post_url = "https://www.instagram.com/p/B_ghUHuFtVr/"
post_url = 'https://www.instagram.com/p/B_liTk6BHLe/'
like_comments(post_url, 276, True)

  0%|          | 0/276 [00:00<?, ?it/s]

[OK] All comments in the screen
3 already loved


  4%|▍         | 12/276 [00:28<10:34,  2.40s/it]


True

# Followers and following
Save current people who you are following and following you

In [89]:
def get_inner_html(elem):
    return elem.get_attribute('innerHTML')

def click_post_followers_following(index):
    '''You shoud be in the target profile before execute the function.
    If index == 0 post else if index == 1 followers else if index == 2 following else error will be raised. '''
    ul_post_followers_following = driver.find_element_by_class_name("k9GMp ")
    li_elems = ul_post_followers_following.find_elements_by_tag_name('li')
    li_selected = li_elems[index]
    li_selected.find_element_by_tag_name('a').click() 
    
    # how many li are there
    return int(li_selected.find_element_by_tag_name('span').text)

def get_nicknames(people_count) -> set:
    '''We'll scrap nicknames by looking followers, following field in our account  
    '''    
    print('Scrolling...')
    div = driver.find_element_by_class_name('isgrP')
    ul_scroll = div.find_element_by_tag_name('ul')
    for _ in tqdm(range(people_count // 12)):  # Insta loads 12 elements per upload
        driver.execute_script('arguments[0].scrollIntoView(false)', ul_scroll)  # scroll all the way down to load all li elements inside ul list
        time.sleep(1)  # wait to load html elements
    
    li_people = driver.find_elements_by_class_name('wo9IH')  # it may find elements inside ul_scoll variable
    people_names = set()
    print('Appending nicknames in list...')
    for li_elem in tqdm(li_people, total=len(li_people)):
        div_elem = li_elem.find_element_by_class_name('d7ByH')
        a_elem = div_elem.find_element_by_tag_name('a')
        people_names.add(a_elem.text)
    return people_names

def __get_people(option, url_profile):
    driver.get(url_profile)
    time.sleep(2)  # call time after update the document
    n = click_post_followers_following(option)
    print(f'Followers: {n}') if option == 1 else print(f'Following: {n}')    
    time.sleep(2)
    return get_nicknames(n)

def get_followers(username):
    url_profile = f'https://www.instagram.com/{username}/'
    return __get_people(1, url_profile)

def get_following(username):
    url_profile = f'https://www.instagram.com/{username}/'
    return __get_people(2, url_profile)


In [90]:
followers = get_followers(username)


Followers: 518



  0%|          | 0/43 [00:00<?, ?it/s]

Scrolling...



100%|██████████| 43/43 [00:45<00:00,  1.05s/it]

  1%|          | 3/518 [00:00<00:20, 24.86it/s]

Appending nicknames in list...



 90%|████████▉ | 464/518 [00:20<00:02, 22.72it/s]


100%|██████████| 518/518 [00:23<00:00, 22.19it/s]


In [91]:
following = get_following(username)

Following: 570



  0%|          | 0/47 [00:00<?, ?it/s]

Scrolling...



100%|██████████| 47/47 [00:52<00:00,  1.11s/it]

  1%|          | 3/570 [00:00<00:23, 24.30it/s]

Appending nicknames in list...



 81%|████████▏ | 464/570 [00:20<00:04, 25.00it/s]


100%|██████████| 570/570 [00:25<00:00, 21.93it/s]


In [92]:
print(len(followers), len(following))
with open('followers.txt', 'w') as fp:
    fp.write('\n'.join(followers))

with open('following.txt', 'w') as fp:
    fp.write('\n'.join(following))

518 570


In [94]:
non_following_back = following - followers
with open('non_following_back.txt', 'w') as fp:
    fp.write('\n'.join(non_following_back))
len(non_following_back)

## Remove non followers

In [107]:

def is_current_private():
    '''We assume we are following him. We click the unfollow button, then read if IG notifies us if it's private'''
    try:
        btn_following = driver.find_element_by_xpath("//span[@aria-label='Following']")
    except NoSuchElementException:
        print('You were not following him')
        return False
    btn_following.click()
    time.sleep(4)
    dialog = driver.find_element_by_xpath("//div[@role='dialog']")
    unfollow_btn, cancel_btn = dialog.find_elements_by_tag_name('button')
    msg = dialog.find_element_by_class_name('_08v79')
    private = 'If you change your mind' in msg.text  # if IG is in English it will work
    cancel_btn.click()
    time.sleep(4)
    return private


In [22]:

def get_user_info(username=None):
    '''Get basic info from users. Window must be maximized in order to work.'''
    # load user's profile
    if username is not None:
        username_url = f'https://www.instagram.com/{username}/'
        driver.get(username_url)
        time.sleep(random.randint(7, 15))

    # find ul elements (required for the count fields)
    ul = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/header/section/ul')
    li_list = ul.find_elements_by_tag_name('li')
    fetch_text = lambda e: e.find_element_by_class_name('g47SY ').text
    
    posts = fetch_text(li_list[0])
    followers = fetch_text(li_list[1])
    following = fetch_text(li_list[2])
    bio_elem =  driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/header/section/div[2]')
    return posts, followers, following, bio_elem.text

In [99]:
# Prepare people to be removed
with open('./txt/non_following_back.txt', 'r') as non_following:
    non_following_set = set(non_following.read().split('\n')[:-1])  # ignore the last element, it's ''
with open('./txt/whitelist.txt', 'r') as whitelist_fp:
    whitelist_set = set(whitelist_fp.read().split('\n')[:-1])
removables = non_following_set - whitelist_set

In [100]:
df = pd.read_csv('./txt/unfollowed.csv')
removables = removables - set(df.nick)

In [105]:

def unfollow_current(wait_dialog_secs=6):
    '''Unfollow the current profile where we are'''
    # unfollow him
    try:
        btn_following = driver.find_element_by_xpath("//span[@aria-label='Following']")
    except NoSuchElementException:
        print('You were not following him')
        return None
    btn_following.click()
    time.sleep(wait_dialog_secs)  # wait dialog
    dialog = driver.find_element_by_xpath("//div[@role='dialog']")
    unfollow_btn, cancel_btn = dialog.find_elements_by_tag_name('button')
    msg = dialog.find_element_by_class_name('_08v79').text
    unfollow_btn.click()
    return 'If you change your mind' in msg  # is private ?



In [106]:


fp = open('./txt/unfollowed.csv', mode='a+')
writer = csv.writer(fp)  # csv ready to append information


In [110]:
bugs = []
for nick in tqdm(removables, total=len(removables), position=0, leave=True):
    # change to user profile
    username_url = f'https://www.instagram.com/{nick}/'
    driver.get(username_url)
    time.sleep(random.randint(15, 20))
    
    
    # verify if user exists
    try:
        h2 = driver.find_element_by_xpath("//h2[@class='VnYfv']")
        print(f"user {nick} not found")
        bugs.append(nick)
        continue
    except NoSuchElementException:
        pass
    
    posts, followers, following, bio = get_user_info()  # you also change to profile
    private = unfollow_current()
    if private is None:
        print(f"user {nick} hadn't been followed")
        bugs.append(nick)
    else:
        writer.writerow([nick, posts, followers, following, bio, private])
        fp.flush()  # important to reflect rows in csv file
fp.close()

100%|██████████| 32/32 [21:55<00:00, 41.10s/it]


In [ ]:
removables = removables - set(bugs)

In [ ]:
print(bugs)

In [111]:
df = pd.read_csv('./csv/people.csv')